# Quick Start

[![DownloadNotebook](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_notebook_en.svg)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/master/mindchemistry/en/quick_start/mindspore_quick_start.ipynb)&emsp;[![DownloadCode](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_download_code_en.svg)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/master/mindchemistry/en/quick_start/mindspore_quick_start.py)&emsp;[![View Source On Gitee](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_source_en.svg)](https://gitee.com/mindspore/docs/blob/master/docs/mindchemistry/docs/source_en/quick_start/quick_start.ipynb)

## Overview

Taking the prediction of interatomic potential with Allegro as an example.

Allegro is a state-of-the-art model built on equivariant graph neural networks. The related paper has been published in the journal Nature Communications. This case study demonstrates the effectiveness of Allegro in molecular potential energy prediction, with high application value.

This tutorial introduces the research background and technical path of Allegro, and demonstrates how to train and perform fast inference with MindSpore Chemistry. More information can be found in [paper](https://www.nature.com/articles/s41467-023-36329-y).

## Technology Path

MindSpore Earth solves the problem as follows:

1. Data Construction.
2. Model Construction.
3. Loss function.
4. Optimizer.
5. Model Training.
6. Model Prediction.

In [1]:
import time
import random

import mindspore as ms
import numpy as np
from mindspore import nn
from mindspore.experimental import optim

The following `src` can be downloaded in [allegro/src](https://gitee.com/mindspore/mindscience/tree/master/MindChemistry/applications/allegro/src).

In [13]:
from mindchemistry.cell.allegro import Allegro
from mindchemistry.utils.load_config import load_yaml_config_from_path

from src.allegro_embedding import AllegroEmbedding
from src.dataset import create_training_dataset, create_test_dataset
from src.potential import Potential

In [3]:
ms.set_seed(123)
ms.dataset.config.set_seed(1)
np.random.seed(1)
random.seed(1)

You can get parameters of model, data and optimizer from [config](https://gitee.com/mindspore/mindscience/blob/master/MindChemistry/applications/allegro/rmd.yaml).

In [4]:
configs = load_yaml_config_from_path("rmd.yaml")
ms.set_context(mode=ms.GRAPH_MODE)
ms.set_device("Ascend", 0)

## Data Construction

In [Revised MD17 dataset (rMD17)](https://gitee.com/link?target=https%3A%2F%2Ffigshare.com%2Farticles%2Fdataset%2FRevised_MD17_dataset_rMD17_%2F12672038), download the dataset to the `./dataset/rmd17/npz_data/` directory. The default configuration file reads the dataset path as `dataset/rmd17/npz_data/rmd17_uracil.npz`.

In [5]:
n_epoch = 5
batch_size = configs['BATCH_SIZE']
batch_size_eval = configs['BATCH_SIZE_EVAL']
learning_rate = configs['LEARNING_RATE']
is_profiling = configs['IS_PROFILING']
shuffle = configs['SHUFFLE']
split_random = configs['SPLIT_RANDOM']
lrdecay = configs['LRDECAY']
n_train = configs['N_TRAIN']
n_eval = configs['N_EVAL']
patience = configs['PATIENCE']
factor = configs['FACTOR']
parallel_mode = "NONE"

print("Loading data...                ")
data_path = configs['DATA_PATH']
ds_train, edge_index, batch, ds_test, eval_edge_index, eval_batch, num_type = create_training_dataset(
    config={
        "path": data_path,
        "batch_size": batch_size,
        "batch_size_eval": batch_size_eval,
        "n_train": n_train,
        "n_val": n_eval,
        "split_random": split_random,
        "shuffle": shuffle
    },
    dtype=ms.float32,
    pred_force=False,
    parallel_mode=parallel_mode
)

Loading data...                


## Model Construction

The Allegro model can be imported using the mindchemistry library, while the Embedding and potential energy prediction modules can be imported from src.

In [6]:
def build(num_type, configs):
    """ Build Potential model

    Args:
        num_atom (int): number of atoms

    Returns:
        net (Potential): Potential model
    """
    literal_hidden_dims = 'hidden_dims'
    literal_activation = 'activation'
    literal_weight_init = 'weight_init'
    literal_uniform = 'uniform'

    emb = AllegroEmbedding(
        num_type=num_type,
        cutoff=configs['CUTOFF']
    )

    model = Allegro(
        l_max=configs['L_MAX'],
        irreps_in={
            "pos": "1x1o",
            "edge_index": None,
            "node_attrs": f"{num_type}x0e",
            "node_features": f"{num_type}x0e",
            "edge_embedding": f"{configs['NUM_BASIS']}x0e"
        },
        avg_num_neighbor=configs['AVG_NUM_NEIGHBOR'],
        num_layers=configs['NUM_LAYERS'],
        env_embed_multi=configs['ENV_EMBED_MULTI'],
        two_body_kwargs={
            literal_hidden_dims: configs['two_body_latent_mlp_latent_dimensions'],
            literal_activation: 'silu',
            literal_weight_init: literal_uniform
        },
        latent_kwargs={
            literal_hidden_dims: configs['latent_mlp_latent_dimensions'],
            literal_activation: 'silu',
            literal_weight_init: literal_uniform
        },
        env_embed_kwargs={
            literal_hidden_dims: configs['env_embed_mlp_latent_dimensions'],
            literal_activation: None,
            literal_weight_init: literal_uniform
        },
        enable_mix_precision=configs['enable_mix_precision'],
    )

    net = Potential(
        embedding=emb,
        model=model,
        avg_num_neighbor=configs['AVG_NUM_NEIGHBOR'],
        edge_eng_mlp_latent_dimensions=configs['edge_eng_mlp_latent_dimensions']
    )

    return net

In [7]:
print("Initializing model...              ")
model = build(num_type, configs)

Initializing model...              


## Loss Function

Allegro uses mean squared error and mean absolute error for model training.

In [8]:
loss_fn = nn.MSELoss()
metric_fn = nn.MAELoss()

## Optimitizer

The Adam optimizer is used, and the learning rate update strategy is ReduceLROnPlateau.

In [9]:
optimizer = optim.Adam(params=model.trainable_params(), lr=learning_rate)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=factor, patience=patience)

## Model Training

In this tutorial, we customize the train_step and test_step, and perform model training.

In [10]:
# 1. Define forward function
def forward(x, pos, edge_index, batch, batch_size, energy):
    pred = model(x, pos, edge_index, batch, batch_size)
    loss = loss_fn(pred, energy)
    if batch_size != 0:
        square_atom_num = (x.shape[0] / batch_size) ** 2
    else:
        raise ValueError("batch_size should not be zero")
    if square_atom_num != 0:
        loss = loss / square_atom_num
    else:
        raise ValueError("square_atom_num should not be zero")
    return loss

# 2. Get gradient function
backward = ms.value_and_grad(forward, None, optimizer.parameters)

# 3. Define function of one-step training and validation
@ms.jit
def train_step(x, pos, edge_index, batch, batch_size, energy):
    loss_, grads_ = backward(x, pos, edge_index, batch, batch_size, energy)
    optimizer(grads_)
    return loss_

@ms.jit
def test_step(x, pos, edge_index, batch, batch_size):
    return model(x, pos, edge_index, batch, batch_size)

def _unpack(data):
    return (data['x'], data['pos']), data['energy']

def train_epoch(model, trainset, edge_index, batch, batch_size, loss_train: list):
    size = trainset.get_dataset_size()
    model.set_train()
    total_train_loss = 0
    loss_train_epoch = []
    ti = time.time()
    for current, data_dict in enumerate(trainset.create_dict_iterator()):
        inputs, label = _unpack(data_dict)
        loss = train_step(inputs[0], inputs[1], edge_index, batch, batch_size, label)
        # AtomWise
        loss = loss.asnumpy()
        loss_train_epoch.append(loss)
        if current % 10 == 0:
            # pylint: disable=W1203
            print(f"loss: {loss:.16f}  [{current:>3d}/{size:>3d}]")
        total_train_loss += loss

    loss_train.append(loss_train_epoch)
    if size != 0:
        loss_train_avg = total_train_loss / size
    else:
        raise ValueError("size should not be zero")
    t_now = time.time()
    print('train loss: %.16f, time gap: %.4f' %(loss_train_avg, (t_now - ti)))

def test(model, dataset, edge_index, batch, batch_size, loss_fn, loss_eval: list, metric_fn, metric_list: list):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    test_loss = 0
    metric = 0
    for _, data_dict in enumerate(dataset.create_dict_iterator()):
        inputs, label = _unpack(data_dict)
        if batch_size != 0:
            atom_num = inputs[0].shape[0] / batch_size
        else:
            raise ValueError("batch_size should not be zero")
        square_atom_num = atom_num ** 2
        pred = test_step(inputs[0], inputs[1], edge_index, batch, batch_size)
        if square_atom_num != 0:
            test_loss += loss_fn(pred, label).asnumpy() / square_atom_num
        else:
            raise ValueError("square_atom_num should not be zero")
        if atom_num != 0:
            metric += metric_fn(pred, label).asnumpy() / atom_num
        else:
            raise ValueError("atom_num should not be zero")

    test_loss /= num_batches
    metric /= num_batches
    # AtomWise
    loss_eval.append(test_loss)
    metric_list.append(metric)
    print("Test: mse loss: %.16f" %test_loss)
    print("Test: mae metric: %.16f" %metric)
    return test_loss

# == Training ==
if is_profiling:
    print("Initializing profiler...      ")
    profiler = ms.Profiler(output_path="dump_output" + "/profiler_data", profile_memory=True)

print("Initializing train...         ")
print("seed is: %d" %ms.get_seed())
loss_eval = []
loss_train = []
metric_list = []
for t in range(n_epoch):
    print("Epoch %d\n-------------------------------" %(t + 1))
    train_epoch(model, ds_train, edge_index, batch, batch_size, loss_train)
    test_loss = test(
        model, ds_test, eval_edge_index, eval_batch, batch_size_eval, loss_fn, loss_eval, metric_fn, metric_list
    )

    if lrdecay:
        lr_scheduler.step(test_loss)
        last_lr = optimizer.param_groups[0].get('lr').value()
        print("lr: %.10f\n" %last_lr)

    if (t + 1) % 50 == 0:
        ms.save_checkpoint(model, "./model.ckpt")

if is_profiling:
    profiler.analyse()

print("Training Done!")

Initializing train...         
seed is: 123
Epoch 1
-------------------------------
loss: 468775552.0000000000000000  [  0/190]
loss: 468785216.0000000000000000  [ 10/190]
loss: 468774752.0000000000000000  [ 20/190]
loss: 468760512.0000000000000000  [ 30/190]
loss: 468342560.0000000000000000  [ 40/190]
loss: 414531872.0000000000000000  [ 50/190]
loss: 435014.9062500000000000  [ 60/190]
loss: 132964368.0000000000000000  [ 70/190]
loss: 82096352.0000000000000000  [ 80/190]
loss: 12417458.0000000000000000  [ 90/190]
loss: 202487.4687500000000000  [100/190]
loss: 300066.6875000000000000  [110/190]
loss: 468295.9375000000000000  [120/190]
loss: 1230706.0000000000000000  [130/190]
loss: 487508.2812500000000000  [140/190]
loss: 242425.6406250000000000  [150/190]
loss: 841241.0000000000000000  [160/190]
loss: 84912.1328125000000000  [170/190]
loss: 1272812.5000000000000000  [180/190]
train loss: 139695450.3818462193012238, time gap: 101.1313
Test: mse loss: 328262.9555555555853061
Test: mae me

## Model Prediction

Define a custom pred function for model prediction and return the prediction results.

In [11]:
def pred(configs, dtype=ms.float32):
    """Pred the model on the eval dataset."""
    batch_size_eval = configs['BATCH_SIZE_EVAL']
    n_eval = configs['N_EVAL']

    print("Loading data...                ")
    data_path = configs['DATA_PATH']
    _, _, _, ds_test, eval_edge_index, eval_batch, num_type = create_test_dataset(
        config={
            "path": data_path,
            "batch_size_eval": batch_size_eval,
            "n_val": n_eval,
        },
        dtype=dtype,
        pred_force=False
    )

    # Define model
    print("Initializing model...              ")
    model = build(num_type, configs)

    # load checkpoint
    ckpt_file = './model.ckpt'
    ms.load_checkpoint(ckpt_file, model)

    # Instantiate loss function and metric function
    loss_fn = nn.MSELoss()
    metric_fn = nn.MAELoss()

    # == Evaluation ==
    print("Initializing Evaluation...         ")
    print("seed is: %d" %ms.get_seed())

    pred_list, test_loss, metric = evaluation(
        model, ds_test, eval_edge_index, eval_batch, batch_size_eval, loss_fn, metric_fn
    )

    print("prediction saved")
    print("Test: mse loss: %.16f" %test_loss)
    print("Test: mae metric: %.16f" %metric)

    print("Predict Done!")

    return pred_list, test_loss, metric


def evaluation(model, dataset, edge_index, batch, batch_size, loss_fn, metric_fn):
    """evaluation"""
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    test_loss = 0
    metric = 0
    pred_list = []
    for _, data_dict in enumerate(dataset.create_dict_iterator()):
        inputs, label = _unpack(data_dict)
        if batch_size != 0:
            atom_num = inputs[0].shape[0] / batch_size
        else:
            raise ValueError("batch_size should not be zero")
        square_atom_num = atom_num ** 2
        prediction = model(inputs[0], inputs[1], edge_index, batch, batch_size)
        pred_list.append(prediction.asnumpy())
        if square_atom_num != 0:
            test_loss += loss_fn(prediction, label).asnumpy() / square_atom_num
        else:
            raise ValueError("square_atom_num should not be zero")
        if atom_num != 0:
            metric += metric_fn(prediction, label).asnumpy() / atom_num
        else:
            raise ValueError("atom_num should not be zero")

    test_loss /= num_batches
    metric /= num_batches

    return pred_list, test_loss, metric

In [14]:
pred(configs)

Loading data...                
Initializing model...              
Initializing Evaluation...         
seed is: 123
prediction saved
Test: mse loss: 901.1434895833332348
Test: mae metric: 29.0822919209798201
Predict Done!


([array([[-259531.56],
         [-259377.28],
         [-259534.83],
         [-259243.62],
         [-259541.62]], dtype=float32),
  array([[-259516.4 ],
         [-259519.81],
         [-259545.69],
         [-259428.45],
         [-259527.28]], dtype=float32),
  array([[-259508.94],
         [-259521.22],
         [-259533.28],
         [-259465.56],
         [-259523.88]], dtype=float32),
  array([[-259533.56],
         [-259303.9 ],
         [-259509.53],
         [-259369.22],
         [-259514.4 ]], dtype=float32),
  array([[-259368.25],
         [-259487.45],
         [-259545.94],
         [-259379.47],
         [-259494.19]], dtype=float32),
  array([[-259533.64],
         [-259453.  ],
         [-259542.69],
         [-259451.9 ],
         [-259213.11]], dtype=float32),
  array([[-259562.5 ],
         [-259531.6 ],
         [-259526.5 ],
         [-259530.3 ],
         [-259389.12]], dtype=float32),
  array([[-259515.03],
         [-259530.69],
         [-259476.9 ],
       